In [1]:
import torch
from torch.cuda.amp import autocast
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from my_utils import data, models
from my_utils.utils import Hook

# life save magic code
%load_ext autoreload
%autoreload 2

In [2]:
Dataset_DIR = '/home/wangyh/01-Projects/03-my/Datasets/polygons_unfilled_64_3_vis'
T = transforms.Compose([
    transforms.ToTensor()
])
vis_data = data.MyDataset(os.path.join(Dataset_DIR, 'test'), transform=T)
vis_loader = DataLoader(vis_data, batch_size=2)

In [3]:
imgs, labels = next(iter(vis_loader))
print(imgs.shape)
print(labels)

torch.Size([2, 1, 64, 64])
tensor([0, 0])


In [11]:
device = "cuda:1"
model = models.simple_Conv(kernel_size=7, depths=(1, 2, 1), dims=(2, 4, 8), act='relu', norm='BN', use_GSP=True)
model.to(device)
print(model)

simple_Conv(
  (act_layer): ReLU()
  (stages): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 2, kernel_size=(7, 7), stride=(1, 1), padding=same, bias=False)
      (BN-2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(2, 4, kernel_size=(7, 7), stride=(1, 1), padding=same, bias=False)
      (BN-4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Sequential(
        (0): Conv2d(4, 4, kernel_size=(7, 7), stride=(1, 1), padding=same, bias=False)
        (BN-4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (3): Sequential(
      (0): Conv2d(4, 8, kernel_size=(7, 7), stride=(1, 1), padding=same, bias=False)
      (BN-8): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (GAP): GAP(
    (avg_pool): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (head): 

In [5]:
len_loader = len(vis_loader)
iter_loader = iter(vis_loader)

In [10]:
iter_loader.dataset

AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'dataset'

In [8]:
for epoch in range(2):
    for step in range(len_loader):
        if step == len_loader-1:
            gap_hook = Hook(verbose=True)
            hook_handle = model.GAP.register_forward_hook(gap_hook)
        try:
            inputs, labels = next(iter_loader)
        except Exception as e:
            print(e)
            continue
        inputs = inputs.to(device)
        labels = labels.to(device)
        batch_size = inputs.size(0)
        with torch.no_grad():
            with autocast():
                _, _ = model((inputs, labels))

    gap_feas = torch.cat(gap_hook.out_features, dim=0)
    print(gap_feas)
    print(labels)
    print(inputs)


NotImplementedError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors, or that you (the operator writer) forgot to register a fallback function.  Available functions are [CPU, CUDA, QuantizedCPU, BackendSelect, Python, Named, Conjugate, Negative, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, UNKNOWN_TENSOR_TYPE_ID, Autocast, Batched, VmapMode].

CPU: registered at aten/src/ATen/RegisterCPU.cpp:18433 [kernel]
CUDA: registered at aten/src/ATen/RegisterCUDA.cpp:26493 [kernel]
QuantizedCPU: registered at aten/src/ATen/RegisterQuantizedCPU.cpp:1068 [kernel]
BackendSelect: fallthrough registered at ../aten/src/ATen/core/BackendSelectFallbackKer

In [14]:
gap_hook.out_features, labels_collector

([], [])

In [24]:
torch.cat(gap_hook.out_features, dim=0), torch.cat(labels_collector, dim=0)

NotImplementedError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors, or that you (the operator writer) forgot to register a fallback function.  Available functions are [CPU, CUDA, QuantizedCPU, BackendSelect, Python, Named, Conjugate, Negative, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, UNKNOWN_TENSOR_TYPE_ID, Autocast, Batched, VmapMode].

CPU: registered at aten/src/ATen/RegisterCPU.cpp:18433 [kernel]
CUDA: registered at aten/src/ATen/RegisterCUDA.cpp:26493 [kernel]
QuantizedCPU: registered at aten/src/ATen/RegisterQuantizedCPU.cpp:1068 [kernel]
BackendSelect: fallthrough registered at ../aten/src/ATen/core/BackendSelectFallbackKer